# Building Model Spacy
### By **Néstor Suat** in 2020

**Descripción:** Generando un modelo ML con Spacy para la tarea de NER en tweets de accidentes para las etiquetas `loc` y `time` usando el estandar BIO.

**Input:**
* TSV con dataset etiquetado con BIO

**Output:**
* Model


    
    python 1\ Building\ Spacy\ NER.py -m es_core_news_lg -o /home/hat/code/traffic-accidents/model/data/v1/NER/spacy_model/ -n 500

***

### Importando librerías

In [9]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split

In [12]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            #s = self.grouped["Sentence: {}".format(self.n_sent)]
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

#### Importando modelo spacy construido localmente

In [3]:
spacy_model = "../../../data/v1/NER/spacy_model/"
nlp = spacy.load(spacy_model)

### Importando dataset anotado

El archivo `ner-crf-training-data.tsv` fue construido anteriormente transformando el formato de anotación de Standoff a BIO.

In [8]:
file = 'ner-crf-training-data.tsv'
dir_ = "../../../data/v1/NER/"
data = pd.read_csv(dir_+file, delimiter = "\t", quoting = 3, names=['Sentence #','Word','POS','Tag'])

In [10]:
data

,Sentence #,Word,POS,Tag
0,13,A,ADP,O
1,13,esta,DET,O
2,13,hora,NOUN,O
3,13,nuestras,DET,O
4,13,unidades,NOUN,O
...,...,...,...,...
1207,42,ha,AUX,O
1208,42,reportado,VERB,O
1209,42,en,ADP,O
1210,42,el,DET,O


In [29]:
getter = SentenceGetter(data)
sentences = getter.sentences
sentences[0][:][0]

('sectormovilidad', 'NOUN', 'O')

### Evaluando Modelo

In [4]:
test_text = 'TM ahora (12:19 p.m.) Si te movilizas por la Troncal Caracas te informamos que se presenta un accidente vehicular a la altura de la estación Calle 57, sentido norte - sur. Nuestra flota debe salir al mixto'
doc2 = nlp(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

B-loc Troncal
I-loc Caracas
B-loc Calle
I-loc 57


In [5]:
test_text = 'Accidente 4 carros Carrera 30 con 13'
doc2 = nlp(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

B-loc Carrera
I-loc 30
I-loc con
